In [ ]:

import pandas as pd
import glob,os
from combat.pycombat import pycombat
import matplotlib.pyplot as plt

_file = "GSE_ALL_hgu133a2hsentrezgcdf.txt"

In [ ]:
# READ CLINICAL FILE, DOWNLOAD FROM GSE73072 repo.
gse = pd.read_csv(_file,sep="\t")
clinic = pd.read_csv("GSE73072_clinical.tsv",sep="\t")


In [ ]:

#BATCH by experiment type
#Since each experiment is stored as a separate dataset, batch normalization was applied.
#For normalization, each experiment was kept within its own VRS file and separated for each experiment using the clinical file information and the GSE data that was read.
#As a result, each experiment was saved separately as a .viruscsv file.
#Afterwards, these were merged, and a batch correction/value was applied based on the differences.



vrss = set(clinic.Experiment.values.tolist())
gsecols = set(gse.columns.values.tolist())
for vrs in vrss:
    _list = []
    cels = clinic[clinic.Experiment == vrs]['CEL'].values.tolist()
    for elem in cels:
        if elem in gsecols:
            _list.append(elem)
    
    gse[_list].to_csv(vrs +".viruscsv",sep="\t")


datasets = [pd.read_csv(k,sep="\t",index_col=0) for k in os.listdir() if k.endswith("viruscsv")]
df_expression = pd.concat(datasets,join="inner",axis=1)

batch = []
for j in range(len(datasets)):
    batch.extend([j for _ in range(len(datasets[j].columns))])

# run pyComBat
df_corrected = pycombat(df_expression,batch)
df_corrected.to_csv("COMBAT_" + _file,sep="\t")

for _file in [k for k in os.listdir() if k.endswith(".viruscsv")]:
    os.remove(_file)

Found 7 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
